In [1]:
import polars as pl
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression

In [2]:
def RMSE(y_hat: pl.Series, y_obs: pl.DataFrame):

    return (((y_hat - y_obs.to_series())**2).sum()/y_obs.shape[0])**0.5

In [3]:
input_matrix = pl.read_excel(
    source="dane_leki.xlsx"
)

In [4]:
input_matrix.head()

__UNNAMED__0,Nazwa,logK HSA,logKCTAB,CATS3D_00_DD,CATS3D_09_AL,CATS3D_00_AA,Zbiór
i64,str,f64,f64,i64,i64,i64,str
1,"""acetaminophen""",-0.79,-0.63,2,0,2,"""t"""
2,"""acetylsalicylic acid""",-0.23,1.22,1,0,4,"""t"""
3,"""bromazepam""",0.38,0.57,1,0,3,"""t"""
4,"""carbamazepine""",0.69,0.68,0,0,3,"""t"""
5,"""chlorpromazine""",1.18,1.5,0,0,2,"""t"""


In [5]:
Y_obs = input_matrix.select(
    pl.nth(2)
)

In [6]:
Y_obs.head()

logK HSA
f64
-0.79
-0.23
0.38
0.69
1.18


In [7]:
descriptors = input_matrix.select(
    pl.nth([3,4,5,6])
)

In [8]:
descriptors.head()

logKCTAB,CATS3D_00_DD,CATS3D_09_AL,CATS3D_00_AA
f64,i64,i64,i64
-0.63,2,0,2
1.22,1,0,4
0.57,1,0,3
0.68,0,0,3
1.5,0,0,2


In [9]:
pca_model = PCA(n_components=4)

In [10]:
pca_model.fit(descriptors)

PCA(n_components=4)

In [11]:
pca_model.explained_variance_

array([4.67581967, 3.27064572, 0.96644472, 0.27022295])

In [12]:
pca_model.components_

array([[ 0.05905699,  0.02081031,  0.64914805,  0.75808048],
       [ 0.30296835, -0.55414496,  0.58475165, -0.50911595],
       [-0.22498561,  0.74897495,  0.4718871 , -0.40711247],
       [ 0.92417743,  0.36266627, -0.11829997,  0.01934869]])

In [19]:
PC = pl.DataFrame(
    pca_model.fit_transform(descriptors),
    schema=[f"PC{i+1}" for i in range(pca_model.n_components_)])

In [20]:
PC

PC1,PC2,PC3,PC4
f64,f64,f64,f64
-2.300697,-0.649955,0.949429,-0.779364
-0.696091,-0.553551,-1.029994,0.606395
-1.492558,-0.241364,-0.476641,-0.013669
-1.506872,0.346107,-1.250365,-0.274675
-2.216526,1.103657,-1.02774,0.463801
…,…,…,…
0.98374,3.79413,1.504934,-0.839315
2.682795,1.400557,0.358198,0.531457
1.407812,-0.971157,0.982119,1.206894


In [17]:
X_training, X_validation, Y_training, Y_validation = train_test_split(
    PC,
    Y_obs,
    test_size=0.33,
    random_state=42
)

In [18]:
X_training

PC1,PC2
f64,f64
1.407812,-0.971157
-0.696091,-0.553551
-2.216526,1.103657
-3.006217,0.128802
-1.492558,-0.241364
…,…
2.739027,0.697231
-0.251225,2.288619
0.711546,-2.789428


In [19]:
Y_training

logK HSA
f64
0.08
-0.23
1.18
-0.42
0.38
…
0.06
2.05
-1.25


In [ ]:
def principal_component_plot(n: int):

    pca_model = PCA(n_components=n)

    PC = pl.DataFrame(
        pca_model.fit_transform(descriptors),
        schema=[f"PC{i+1}" for i in range(pca_model.n_components_)]
    )

    KFold_model = KFold(
        n_splits=10,
        shuffle=True,
        random_state=0
    )

    X_training, _, Y_training, _ = train_test_split(
        PC,
        Y_obs,
        test_size=0.33,
        random_state=42
    )

    validation_sets = [
        validation_set for (_, validation_set) in KFold_model.split(X_training, Y_training)
    ]

    X=[
        X_training.with_row_index().filter(
            ~pl.col("index").is_in(validation_set)
        ).drop(
            pl.col("index")
        ) for validation_set in validation_sets
    ]

    Y=[
        Y_training.with_row_index().filter(
            ~pl.col("index").is_in(validation_set)
        ).drop(
            pl.col("index")
        ) for validation_set in validation_sets
    ]

    PCR_models = [
        LinearRegression().fit(
            X=x,
            y=y
        ) for (x,y) in zip(X,Y)
    ]

        
    residues = [
        (PCR_models[idx].predict(
            X_training.with_row_index().filter(
            pl.col("index").is_in(validation_sets[idx])
        ).drop(
            pl.col("index")
        )
        ).reshape(-1) - Y_training.with_row_index().filter(
            ~pl.col("index").is_in(validation_sets[idx])
        ).drop(
            pl.col("index")
        )) for idx in range(len(validation_sets))
    ]
    
    print(residues)
    print(residues[0].shape)

    

In [31]:
principal_component_plot(4)

[array([[ 0.30838227,  2.12834894],
       [-0.79161773,  1.02834894],
       [ 0.80838227,  2.62834894],
       [ 0.00838227,  1.82834894],
       [ 0.98838227,  2.80834894],
       [-0.30161773,  1.51834894],
       [ 0.18838227,  2.00834894],
       [-1.43161773,  0.38834894],
       [-1.90161773, -0.08165106],
       [ 1.63838227,  3.45834894],
       [-1.30161773,  0.51834894],
       [ 0.32838227,  2.14834894],
       [-1.66161773,  0.15834894],
       [ 1.63838227,  3.45834894],
       [ 1.63838227,  3.45834894],
       [-0.69161773,  1.12834894]]), array([[ 0.41455754,  1.47717266],
       [ 0.72455754,  1.78717266],
       [-0.68544246,  0.37717266],
       [ 0.91455754,  1.97717266],
       [ 0.11455754,  1.17717266],
       [ 1.09455754,  2.15717266],
       [-1.34544246, -0.28282734],
       [-0.19544246,  0.86717266],
       [-1.32544246, -0.26282734],
       [ 1.74455754,  2.80717266],
       [-1.19544246, -0.13282734],
       [ 0.43455754,  1.49717266],
       [-1.555442

AttributeError: 'list' object has no attribute 'shape'

In [20]:
KFold_model = KFold(
    n_splits=10,
    shuffle=True,
    random_state=0
)

In [21]:
validation_sets = []
for training_set, validation_set in KFold_model.split(X_training, Y_training):
    validation_sets.append(validation_set)
    print(f"Training: {training_set}\nValidation: {validation_set}")

Training: [ 0  2  3  4  5  7  8  9 10 11 12 13 14 15 16 17]
Validation: [1 6]
Training: [ 0  1  2  3  4  5  6  7  9 11 12 13 14 15 16 17]
Validation: [ 8 10]
Training: [ 0  1  2  3  5  6  7  8  9 10 11 12 13 15 16 17]
Validation: [ 4 14]
Training: [ 0  1  3  4  5  6  7  8  9 10 11 12 13 14 15 17]
Validation: [ 2 16]
Training: [ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 15 16]
Validation: [ 9 17]
Training: [ 0  1  2  3  4  5  6  8  9 10 11 12 14 15 16 17]
Validation: [ 7 13]
Training: [ 0  1  2  4  5  6  7  8  9 10 12 13 14 15 16 17]
Validation: [ 3 11]
Training: [ 1  2  3  4  6  7  8  9 10 11 12 13 14 15 16 17]
Validation: [0 5]
Training: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 16 17]
Validation: [15]
Training: [ 0  1  2  3  4  5  6  7  8  9 10 11 13 14 15 16 17]
Validation: [12]


In [22]:
descriptors

logKCTAB,CATS3D_00_DD,CATS3D_09_AL,CATS3D_00_AA
f64,i64,i64,i64
-0.63,2,0,2
1.22,1,0,4
0.57,1,0,3
0.68,0,0,3
1.5,0,0,2
…,…,…,…
0.73,0,5,2
1.63,1,4,5
1.32,3,2,5


In [71]:
LinearRegression().fit(
            X=X_training.with_row_index().filter(~pl.col("index").is_in(validation_set)).drop(pl.col("index")),
            y=Y_training.with_row_index().filter(~pl.col("index").is_in(validation_set)).drop(pl.col("index"))
    )

LinearRegression()

In [86]:
pl.Series(prediction[i][0] for i in range(len(prediction)))

""
f64
0.159224
2.714788


In [76]:
Y_training.with_row_index().filter(pl.col("index").is_in(validation_set)).drop(pl.col("index"))

logK HSA
f64
-0.23
1.84


In [88]:
#predictions = pl.DataFrame()
predictions = []
residues = []
RMSEs = []


for idx, validation_set in enumerate(validation_sets):
    x=X_training.with_row_index().filter(~pl.col("index").is_in(validation_set)).drop(pl.col("index"))
    y=Y_training.with_row_index().filter(~pl.col("index").is_in(validation_set)).drop(pl.col("index"))
    PCR_model = LinearRegression().fit(
            X=x,
            y=y
    )
    prediction = PCR_model.predict(X_training.with_row_index().filter(pl.col("index").is_in(validation_set)).drop(pl.col("index")))

    
    RMSEs.append(
        RMSE(
            pl.Series(prediction[i][0] for i in range(len(prediction))),
            Y_training.with_row_index().filter(pl.col("index").is_in(validation_set)).drop(pl.col("index"))
        )
    )

print(sum(RMSEs))
    

5.181391908438678


In [70]:
x

""
"array[f64, 1]"
[0.159224]
[2.714788]
[0.655465]
[1.780267]
[0.185102]
…
[-1.522106]
[-0.054193]
[-1.08324]


In [68]:
predictions

[shape: (2,)
 Series: '' [array[f64, 1]]
 [
 	[0.159224]
 	[2.714788]
 ],
 shape: (2,)
 Series: '' [array[f64, 1]]
 [
 	[0.655465]
 	[1.780267]
 ],
 shape: (2,)
 Series: '' [array[f64, 1]]
 [
 	[0.185102]
 	[1.548124]
 ],
 shape: (2,)
 Series: '' [array[f64, 1]]
 [
 	[0.845459]
 	[-1.10008]
 ],
 shape: (2,)
 Series: '' [array[f64, 1]]
 [
 	[1.208883]
 	[0.583907]
 ],
 shape: (2,)
 Series: '' [array[f64, 1]]
 [
 	[0.51657]
 	[1.110712]
 ],
 shape: (2,)
 Series: '' [array[f64, 1]]
 [
 	[0.588338]
 	[-1.522106]
 ],
 shape: (2,)
 Series: '' [array[f64, 1]]
 [
 	[-0.054193]
 	[-1.08324]
 ],
 shape: (1,)
 Series: '' [array[f64, 1]]
 [
 	[-0.931429]
 ],
 shape: (1,)
 Series: '' [array[f64, 1]]
 [
 	[1.030242]
 ]]

In [46]:
residues.head(2)

""
object
"shape: (2, 1) ┌───────────┐ │ logK HSA │ │ --- │ │ f64 │ ╞═══════════╡ │ -0.389224 │ │ -0.874788 │ └───────────┘"
"shape: (2, 1) ┌───────────┐ │ logK HSA │ │ --- │ │ f64 │ ╞═══════════╡ │ -0.455465 │ │ 0.509733 │ └───────────┘"
